In [1]:
import netCDF4 as net
import numpy   as np
import pandas  as pd
import xarray  as xr
import matplotlib.pyplot as plt
import shutil
import urllib.request as request
from contextlib import closing
from ftplib import FTP 
from datetime import datetime
import os


systempath = "C:/Users/adria/Desktop/Uni/Semester5/Geosoft2/Code/Notebooks/SST/Data/"

In [2]:
def download_file(year):
    '''
    Downloads the sst data file for the given year
    
    Parameters:
        year (int): The year the sst is needed
   '''
    start = datetime.now()
    ftp = FTP('ftp.cdc.noaa.gov')
    ftp.login()
    ftp.cwd('/Projects/Datasets/noaa.oisst.v2.highres/')

    files = ftp.nlst()
    counter = 0

    for file in files:
        if file == 'sst.day.mean.'+ str(year)+'.nc':
            print("Downloading..." + file)
            ftp.retrbinary("RETR " + file, open(systempath + file, 'wb').write)      
            ftp.close()
            end = datetime.now()
            diff = end - start
            print('All files downloaded for ' + str(diff.seconds) + 's')
        else: counter += 1
    
        if counter == len(files):
            print('No matching dataset found for this year')

In [3]:
def merge_datacubes(ds_merge):
    '''
    merges datacubes by coordinates
    
    Parameters:
        ds_merge (array): array of datasets to be mearched
        
    Returns: 
        ds1 (ds): A single datacube with all merged datacubes
        - Error, if no Datacubes given
    '''
    start = datetime.now()
    if len(ds_merge) == 0:
        print("error")
        return
    if len(ds_merge) == 1:
        return ds_merge[0]
    else:
        print('start merging')
        ds1 = ds_merge[0]
        count = 1
        while count < len(ds_merge):
            start1 = datetime.now()
            ds1 =  xr.combine_by_coords([ds1,ds_merge[count]])
            count+=1
            print("succesfully merged cube nr "+ str(count)+" to the base cube ")
            end = datetime.now()
            diff = end - start1
            print('All cubes merged for ' + str(diff.seconds) + 's')
        print("result: ")
        print(ds1)
        end = datetime.now()
        diff = end - start
        print('All cubes merged for ' + str(diff.seconds) + 's')
        return ds1

In [4]:
def timeframe(ds,start,end):
    '''
    Slices Datacube down to given timeframe
      
    Parameters:
        ds (ds): source dataset
        start (str): start of the timeframe eg '2018-07-13'
        end (str): end of the timeframe eg '2018-08-23'
       
    Returns:
        ds_selected (ds): dataset sliced to timeframe
    '''
    if start>end:
        print("start and end of the timeframe do are not compatible!")
    else:    
        ds_selected = ds.sel(time = slice(start,end))
        #print(ds_selected)
        return ds_selected    

In [5]:
def safe_datacube(ds, timeframe):
    '''
    Saves the Datacube as NetCDF (.nc)
      
    Parameters:
        ds (ds): source dataset
        timeframe (str): timeframe eg '2017', '2015_2019'
    '''
    print("start saving")
    if type(timeframe) != str:
        timeframe=str(timeframe)
    ds.to_netcdf(systempath+ "sst.day.mean."+timeframe+".nc")
    print("done saving")

Example

In [6]:
# download_file(2018)
# download_file(2019)
ds_2018 = xr.open_dataset(systempath+"sst.day.mean.2018.nc")
ds_2019 = xr.open_dataset(systempath+"sst.day.mean.2019.nc")
ds_merge = [ds_2018,ds_2019]
# ds_2018_2019 = merge_datacubes(ds_merge)
ds_2018_2019 = xr.open_dataset(systempath+"sst.day.mean.2018-2019.nc")
ds_sliced = timeframe(ds_2018_2019,'2018-07-01','2019-08-01')
print(ds_sliced.time)
# safe_datacube(ds_2018_2019,"201819")

<xarray.DataArray 'time' (time: 397)>
array(['2018-07-01T00:00:00.000000000', '2018-07-02T00:00:00.000000000',
       '2018-07-03T00:00:00.000000000', ..., '2019-07-30T00:00:00.000000000',
       '2019-07-31T00:00:00.000000000', '2019-08-01T00:00:00.000000000'],
      dtype='datetime64[ns]')
Coordinates:
  * time     (time) datetime64[ns] 2018-07-01 2018-07-02 ... 2019-08-01
Attributes:
    long_name:   Time
    delta_t:     0000-00-01 00:00:00
    avg_period:  0000-00-01 00:00:00
    axis:        T


In [7]:
print(ds_2018)

<xarray.Dataset>
Dimensions:  (lat: 720, lon: 1440, time: 365)
Coordinates:
  * time     (time) datetime64[ns] 2018-01-01 2018-01-02 ... 2018-12-31
  * lat      (lat) float32 -89.88 -89.62 -89.38 -89.12 ... 89.38 89.62 89.88
  * lon      (lon) float32 0.125 0.375 0.625 0.875 ... 359.1 359.4 359.6 359.9
Data variables:
    sst      (time, lat, lon) float32 ...
Attributes:
    Conventions:    CF-1.5
    title:          NOAA/NCEI 1/4 Degree Daily Optimum Interpolation Sea Surf...
    institution:    NOAA/National Centers for Environmental Information
    source:         NOAA/NCEI https://www.ncei.noaa.gov/data/sea-surface-temp...
    References:     https://www.psl.noaa.gov/data/gridded/data.noaa.oisst.v2....
    dataset_title:  NOAA Daily Optimum Interpolation Sea Surface Temperature
    version:        Version 2.1
    comment:        Reynolds, et al.(2007) Daily High-Resolution-Blended Anal...
